### setting up dependencies from corenet

In [46]:
from pycrate_core.repr          import *
from pycrate_corenet.utils      import *
from pycrate_corenet.Server     import CorenetServer
from pycrate_corenet.ServerGTPU import ARPd, GTPUd, MOD, DNSRESP, TCPSYNACK, BLACKHOLE_LAN, BLACKHOLE_WAN
from pycrate_corenet.ServerAuC  import AuC
from pycrate_corenet.ServerSMS  import SMSd
from pycrate_corenet.HdlrHNB    import HNBd
from pycrate_corenet.HdlrENB    import ENBd
from pycrate_corenet.HdlrUE     import UEd
from pycrate_corenet.HdlrUEIuCS import UEIuCSd, UEMMd, UECCd, UESSd
from pycrate_corenet.HdlrUEIuPS import UEIuPSd, UEGMMd, UESMd
from pycrate_corenet.HdlrUESMS  import UESMSd
from pycrate_corenet.HdlrUES1   import UES1d, UEEMMd, UEESMd
from pycrate_corenet import ProcCNHnbap
from pycrate_corenet import ProcCNRua
from pycrate_corenet import ProcCNRanap
from pycrate_corenet import ProcCNS1ap
from pycrate_corenet import ProcCNMM
from pycrate_corenet import ProcCNSMS
from pycrate_corenet import ProcCNGMM
from pycrate_corenet import ProcCNSM
from pycrate_corenet import ProcCNEMM
from pycrate_corenet import ProcCNESM
import socket
import os
CorenetServer.DEBUG   = ('ERR', 'WNG', 'INF', 'DBG')
AuC.DEBUG             = ('ERR', 'WNG', 'INF', 'DBG')
ARPd.DEBUG            = ('ERR', 'WNG', 'INF', 'DBG')
GTPUd.DEBUG           = ('ERR', 'WNG', 'INF', 'DBG')
SMSd.DEBUG            = ('ERR', 'WNG', 'INF', 'DBG')

### Setting up the binding IP address for the MME and GTP endpoints

In [47]:
LOCAL_IP=socket.gethostbyname(socket.gethostname())
CorenetServer.SERVER_ENB['IP']   = LOCAL_IP
CorenetServer.SERVER_ENB['port'] = 36412 # SCTP port for S1AP protocol
CorenetServer.SERVER_ENB['GTPU'] = LOCAL_IP

CorenetServer.SERVER_HNB['IP']   = LOCAL_IP
CorenetServer.SERVER_HNB['port'] = 29169 # SCTP port for HNBAP and RUA protocols

### external interfaces (toward Internet or other service networks)

# ARPd interface info
#ARPd.GGSN_ETH_IF     = 'eth0'
#ARPd.GGSN_MAC_ADDR   = '08:00:00:01:02:03'
#ARPd.GGSN_IP_ADDR    = '192.168.1.100'
# ARPd LAN prefix and router info
#ARPd.SUBNET_PREFIX   = '192.168.1.0/24'
#ARPd.ROUTER_MAC_ADDR = 'f4:00:00:01:02:03'
#ARPd.ROUTER_IP_ADDR  = '192.168.1.1'
## set of IP addresses dedicated to UEs
#ARPd.IP_POOL = set()
#for i in range(149, 250):
#    ARPd.IP_POOL.add('192.168.1.%i' % i)

## GTPUd interface info
#GTPUd.EXT_IF         = ARPd.GGSN_ETH_IF
#GTPUd.EXT_MAC_ADDR   = ARPd.GGSN_MAC_ADDR
## IPv6 network prefix
#GTPUd.EXT_IPV6_PREF  = '2001:123:456:abcd'
## GTPUd BLACKHOLING feature
## to enable the blackholing of UE LAN / WAN traffic, set the appropriate flag(s)
#GTPUd.BLACKHOLING    = 0
#GTPUd.WL_ACTIVE      = False
##GTPUd.BLACKHOLING    = BLACKHOLE_WAN
##GTPUd.BLACKHOLING    = BLACKHOLE_LAN | BLACKHOLE_WAN
##GTPUd.WL_ACTIVE      = True
##GTPUd.WL_PORTS       = [('UDP', 53), ('UDP', 123)]
## GTPUd to generate statistics for each UE's source IP address
#GTPUd.DPI            = True
## GTPUd to drop uplink packet with spoofed source IP address
##GTPUd.DROP_SPOOF     = True
GTPUd.GTP_PORT       = 2152 # UDP port for GTP-U protocol
GTPUd.GTP_IF         = (CorenetServer.SERVER_HNB['GTPU'],
                        CorenetServer.SERVER_ENB['GTPU'], )

### Authentication of UEs

In [48]:
AuC.OP = b'63bfa50ee6523365ff14c1f45f88737d'
AuC.AUC_DB_PATH = "/root/corenet/"
CorenetServer.ConfigUE = {
    '001010000000001': {
        'USIM'  : True,
        'MSISDN': '012345',
        'PDP'   : [('*', 1, '192.168.1.151'),
                   ('corenet', 3, '192.168.1.151', GTPUd.EXT_IPV6_PREF + ':0:1:0:97')],
        'PDN'   : [('*', 1, '192.168.1.151'),
                   ('corenet', 3, '192.168.1.151', '0:1:0:97')]
        },
    '001010000000002': {
        'USIM'  : True,
        'MSISDN': '012346',
        'PDP'   : [('*', 1, '192.168.1.152'),
                   ('corenet', 3, '192.168.1.152', GTPUd.EXT_IPV6_PREF + ':0:1:0:98')],
        'PDN'   : [('*', 1, '192.168.1.152'),
                   ('corenet', 3, '192.168.1.152', '0:1:0:98')]
        }
}
CorenetServer.UE_ATTACH_FILTER = None
CorenetServer.ConfigUE['*'] = {
        'USIM'  : True,
        'MSISDN': '000111',
        'PDP'   : [('*', 1, '192.168.1.149'),
                   ('corenet', 3, '192.168.1.149', GTPUd.EXT_IPV6_PREF + ':0:1:0:95')],
        'PDN'   : [('*', 1, '192.168.1.149'),
                   ('corenet', 3, '192.168.1.149', '0:1:0:95')]
}

### PLMN and core network parameters

In [49]:
CorenetServer.PLMN = '72207'
CorenetServer.EQUIV_PLMN = None
CorenetServer.RAN = {}
CorenetServer.RAN_CONNECT_ANY = True
CorenetServer.RAN_ALLOWED_PLMN = [CorenetServer.PLMN]
CorenetServer.EMERG_NUMS = None
CorenetServer.MME_GID  = 1
CorenetServer.MME_CODE = 1
CorenetServer.ConfigS1 = {
    'MMEname': 'CorenetMME',
    'ServedGUMMEIs' : [
        {'servedPLMNs'   : [plmn_str_to_buf(CorenetServer.PLMN)],
         'servedGroupIDs': [uint_to_bytes(CorenetServer.MME_GID, 16)],
         'servedMMECs'   : [uint_to_bytes(CorenetServer.MME_CODE, 8)]}
        ],
    'RelativeMMECapacity': 10,
    'EquivPLMNList'      : CorenetServer.EQUIV_PLMN,
    'EmergNumList'       : CorenetServer.EMERG_NUMS,
}
CorenetServer.ConfigPDN = {
    'corenet': {
        'QCI': 9,
        'DNS': ((1, '8.8.8.8'), # Google DNS servers
                (1, '8.8.4.4'),
                (2, '2001:4860:4860::8888'),
                (2, '2001:4860:4860::8844')),
        'MTU': (None, None),
        },
}
CorenetServer.ConfigPDN['*'] = {
        'QCI': 9,
        'DNS': ((1, '8.8.8.8'), # Google DNS servers
                (1, '8.8.4.4'),
                (2, '2001:4860:4860::8888'),
                (2, '2001:4860:4860::8844')),
        'MTU': (None, None),
}

In [50]:
Server = CorenetServer()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pycrate-0.3-py3.6.egg/pycrate_corenet/Server.py", line 368, in _start_hnb_server
    self._sk_hnb.bind(server_addr)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/pycrate-0.3-py3.6.egg/pycrate_corenet/Server.py", line 311, in start
    self._start_hnb_server()
  File "/usr/local/lib/python3.6/dist-packages/pycrate-0.3-py3.6.egg/pycrate_corenet/Server.py", line 371, in _start_hnb_server
    .format(server_addr, err)))
pycrate_corenet.utils.CorenetErr: cannot bind SCTP socket on address ('172.18.0.2', 29169): [Errno 98] Address already in use

